In [82]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.datasets import fetch_openml
import pickle
from scipy.special import expit
from sklearn.model_selection import train_test_split

## Open MNIST data and create a subset with desired digits

<font color=red>Change the code below to select a different subset </font>

In [83]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
y = np.asarray([int(numeric_string) for numeric_string in y])

# Saving the temporary variables for fast retrieval
with open('temp.pickle', 'wb') as handle:
    pickle.dump([X, y], handle)


In [84]:
# loading the temporary variables for fast retrieval
with open('temp.pickle', 'rb') as handle:
    X, y = pickle.load(handle)

X = X
    
Nclasses = 3
labelclasses = y<Nclasses
Xnew = X[labelclasses]
ynew = y[labelclasses]
Nfeatures = np.size(Xnew,1)
Nsamples = np.size(Xnew,0)


In [85]:
 
X_train, X_test, y_train, y_test = train_test_split(Xnew, ynew, test_size=0.3, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.14, random_state=1) # 0.25 x 0.8 = 0.2

Nsamples = X_train.shape[0]
Nfeatures = X_train.shape[1]

X_train = X_train.T
X_val = X_val.T
X_test = X_test.T


### The Cell Below Contains Basic Utilities Shared Between Steps

In [128]:
def getClassRange(y_data):
    """get class range

    Given a series of class values, since our classes map directly to index values
    (0, 1, 2, 3) we take the max and min value of the series to determine what classes
    are included in our data.
    """
    return range(np.min(y_data), np.max(y_data)+1)

def testLinearMCClassifier(A, x_test, y_expect, output = False):
    misclassifications = 0
    total = y_expect.size

    x_tilde = np.vstack((x_test, np.ones(len(x_test[0]))))
    y_predict = [] #index represents class predictions

    for i in getClassRange(y_expect):
        a_i = A[i]
        y_predict.append(np.matmul(a_i.T, x_tilde))

    for i, expect_y in enumerate(y_expect):
        y_i = y_predict[expect_y]
        if y_i[i] <= 0:
            misclassifications += 1

    if output:
        print(f"Mis-classifications = {misclassifications} out of {total} equivalent to {(misclassifications / total)*100} %")

    return misclassifications


### Classifiers:

In [129]:
def gaussianMultiChannelClassifier(x_data, y_data): # Phase2 - step 2
    """gaussian multi channel classifier

    The following computes the multi-channel gaussian classifier with the
    identity covariance.

    Parameters
    ----------
    x_data
        The input data set values
    y_data
        The associated class for a given 784 pixel grid

    Returns
    -------
    np.array
        A (n x weights) array representing the a_i for a given class in each column
        a_i is used to compute discriminant with a_i * 

    """
    i_cov = np.identity(len(x_data)) #Identity covariance matrix, reused for each class
    inv_i_cov = np.linalg.inv(i_cov)
    result_a = []

    for class_i in getClassRange(y_data):
        class_i_indices = (y_data == class_i)
        x_i = x_data.T[class_i_indices]
        u_i = np.mean(x_i, axis=0)
    
        W_i = 2*np.matmul(inv_i_cov, u_i)
        const_w_i = np.matmul(u_i.T, np.matmul(inv_i_cov, u_i))

        a_i = np.hstack((W_i.T, -const_w_i))
        result_a.append(a_i)

    result_a = np.array(result_a)
    return result_a

# Example usage: gaussianMultiChannelClassifier(X_train, y_train)
A = gaussianMultiChannelClassifier(X_train, y_train)

result = testLinearMCClassifier(A, X_train, y_train, True)
print(result)

Mis-classifications= 207 out of 13105 equivalent to 1.5795497901564288 %
207
